## Programação Linear e Inteira
### Professor Luciano Monteiro
### Cristiana Couto

In [2]:
using JuMP, GLPK

In [32]:
model = Model(GLPK.Optimizer)

#Definindo cores verde = 1, azul = 2, vermelho = 3, 
#azul claro = 4, amarelo = 5, laranja = 6

nodesInput = [1,3,5,6,7,8,9,19,20,21,29,31]

@variable(model, 1 >= N[1:36, 1:4, 1:6] >= 0, Int)
@variable(model, 1 >= C[1:36, 1:6] >= 0, Int)

#Cores do input nos nós terminais
@constraint(model, sum(N[1, i, 6] for i in 1:4) == 1)
@constraint(model, sum(N[3, i, 5] for i in 1:4) == 1)
@constraint(model, sum(N[5, i, 2] for i in 1:4) == 1)
@constraint(model, sum(N[6, i, 1] for i in 1:4) == 1)
@constraint(model, sum(N[7, i, 4] for i in 1:4) == 1)
@constraint(model, sum(N[8, i, 6] for i in 1:4) == 1)
@constraint(model, sum(N[9, i, 3] for i in 1:4) == 1)
@constraint(model, sum(N[19, i, 5] for i in 1:4) == 1)
@constraint(model, sum(N[20, i, 4] for i in 1:4) == 1)
@constraint(model, sum(N[21, i, 3] for i in 1:4) == 1)
@constraint(model, sum(N[29, i, 2] for i in 1:4) == 1)
@constraint(model, sum(N[31, i, 1] for i in 1:4) == 1)

#Cada nó só tem no maxímo 2 arestas saindo dele e no mínimo 1
#Nós terminais só possuem uma aresta saindo dele

for i in 1:36
    if i in nodesInput
        @constraint(model, sum(N[i,1:4, 1:6]) == 1)
    else
        @constraint(model, 1 <= sum(N[i,1:4, 1:6]) <= 2)
    end
end

#Cada aresta só possui uma cor e soma zero se ela não existe
for i in 1:36
    for j in 1:4
       @constraint(model, sum(N[i,j, 1:6]) <= 1)
    end
end

#Os nós da borda não possuem algumas arestas
#1= esquerda, 2= p/ cima, 3= direita, 4 = p/ baixo

#Começando pelos nós que não têm aresta esquerda
for i in 0:5
    @constraint(model, sum(N[i*6 + 1,1, 1:6]) == 0)
end

#Nós que não têm aresta p/ cima
for i in 1:6
    @constraint(model, sum(N[i, 2, 1:6]) == 0)
end

#Nós que não têm aresta p/ direita
for i in 1:6
    @constraint(model, sum(N[i*6, 3, 1:6]) == 0)
end

#Nós que não têm aresta p/ baixo
for i in 31:36
    @constraint(model, sum(N[i, 4, 1:6]) == 0)
end

#A aresta (u,v) tem a mesma cor que a aresta (v,u)
#Os vizinhos na horizontal e vertical que compartilham arestas precisam 
#ter as mesmas cores (começamos pelo subtabuleiro que possui todos os vizinhos)
for i in 8:29
    @constraint(model, N[i, 1, 1:6] .== N[i - 1, 3, 1:6])
    @constraint(model, N[i, 2, 1:6] .== N[i - 6, 4, 1:6])
    @constraint(model, N[i, 3, 1:6] .== N[i + 1, 1, 1:6])
    @constraint(model, N[i, 4, 1:6] .== N[i + 6, 2, 1:6])
end

#Última linha (que não é canto)
for i in 32:35
    @constraint(model, N[i, 1, 1:6] .== N[i - 1, 3, 1:6])
    @constraint(model, N[i, 2, 1:6] .== N[i - 6, 4, 1:6])
    @constraint(model, N[i, 3, 1:6] .== N[i + 1, 1, 1:6])
end

#Primeira linha (que não é canto)
for i in 2:5
    @constraint(model, N[i, 1, 1:6] .== N[i - 1, 3, 1:6])
    @constraint(model, N[i, 3, 1:6] .== N[i + 1, 1, 1:6])
    @constraint(model, N[i, 4, 1:6] .== N[i + 6, 2, 1:6])
end

#Primeira coluna
for i in 1:4
    @constraint(model, N[i*6 + 1, 2, 1:6] .== N[i*6 - 5, 4, 1:6])
    @constraint(model, N[i*6 + 1, 3, 1:6] .== N[i*6 + 2, 1, 1:6])
    @constraint(model, N[i*6 + 1, 4, 1:6] .== N[i*6 + 7, 2, 1:6])
end

#Última coluna
for i in 2:5
    @constraint(model, N[i*6, 1, 1:6] .== N[i*6 - 1, 3, 1:6])
    @constraint(model, N[i*6, 2, 1:6] .== N[i*6 - 6, 4, 1:6])
    @constraint(model, N[i*6, 4, 1:6] .== N[i*6 + 6, 2, 1:6])
end

#Cantos
#Casa [1,1] do tabuleiro
@constraint(model, N[1, 3, 1:6] .== N[2, 1, 1:6])
@constraint(model, N[1, 4, 1:6] .== N[7, 2, 1:6])

#Casa [1,6]
@constraint(model, N[6, 1, 1:6] .== N[5, 3, 1:6])
@constraint(model, N[6, 4, 1:6] .== N[12, 2, 1:6])

#Casa [6,1]
@constraint(model, N[31, 2, 1:6] .== N[25, 4, 1:6])
@constraint(model, N[31, 3, 1:6] .== N[32, 1, 1:6])

#Casa [6,6]
@constraint(model, N[36, 1, 1:6] .== N[35, 3, 1:6])
@constraint(model, N[36, 2, 1:6] .== N[30, 4, 1:6])

#Cada casa do tabuleiro só possui uma cor (cada nó só tem uma cor)
#Se há uma aresta azul saindo do nó u, não pode haver uma aresta laranja chegando em u

#No caso dos nós terminais não tem problema porque só sai uma aresta dele
#Então a cor é fixa e foi determinada nas restrições acima
#O problema é quando podem sair duas arestas de um nó e acabamos com duas arestas 
#uma de cada cor

#Uma possibilidade seria multiplicar as combinações dois a dois dos vetores de arestas 
#de modo que a soma total seria 1 se todas as arestas tivessem a mesma cor,
#mas não seria uma restrição linear

#Vamos utilizar uma matriz 36 x 6 para fixar exatamente uma cor de cada casa no tabuleiro

for i in 1:36
    @constraint(model, sum(C[i, 1:6]) == 1)
    if i ∉ nodesInput
        for j in 1:6
            @constraint(model, C[i,j] == sum(N[i,1:4,j])*0.5)
        end
    else
        for k in 1:6
            @constraint(model, C[i,k] == sum(N[i,1:4,k]))
        end
    end
end


In [75]:
# Chamada do Solver
optimize!(model)

 
#Cores verde = 1, azul = 2, vermelho = 3, azul claro = 4, amarelo = 5, laranja = 6

@show objective_value(model)
value.(N[1,:,:])

objective_value(model) = 0.0


4×6 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0

In [55]:
findall(x->x==1, value.(N))

48-element Array{CartesianIndex{3},1}:
 CartesianIndex(2, 1, 1)
 CartesianIndex(4, 1, 1)
 CartesianIndex(6, 1, 1)
 CartesianIndex(10, 1, 1)
 CartesianIndex(11, 1, 1)
 CartesianIndex(12, 1, 1)
 CartesianIndex(13, 1, 1)
 CartesianIndex(14, 1, 1)
 CartesianIndex(15, 1, 1)
 CartesianIndex(16, 1, 1)
 CartesianIndex(17, 1, 1)
 CartesianIndex(18, 1, 1)
 CartesianIndex(22, 1, 1)
 ⋮
 CartesianIndex(5, 1, 2)
 CartesianIndex(29, 1, 2)
 CartesianIndex(6, 2, 2)
 CartesianIndex(31, 2, 2)
 CartesianIndex(9, 1, 3)
 CartesianIndex(21, 1, 3)
 CartesianIndex(7, 1, 4)
 CartesianIndex(20, 1, 4)
 CartesianIndex(3, 1, 5)
 CartesianIndex(19, 1, 5)
 CartesianIndex(1, 1, 6)
 CartesianIndex(8, 1, 6)

In [ ]:
model = Model(GLPK.Optimizer)
#Quero pintar as arestas, todas as arestas q saem de um nó tem a mesma cor, não pode
#haver cruzamentos

#Definindo cores como números, verde = 1, azul = 2, vermelho = 3, azul claro = 4, amarelo = 5, laranja = 6
@variable(model, 6 >= M[1:36] >= 1, Int)
@variable(model, 6 >= N[1:36, 1:36] >= 0, Int)
@variable(model, 1 >= Y[1:36, 1:36] >= 0, Int)

#
@constraint(model, M[1] == 6)
@constraint(model, M[3] == 5)
@constraint(model, M[5] == 2)
@constraint(model, M[6] == 1)
@constraint(model, M[7] == 4)
@constraint(model, M[8] == 6)
@constraint(model, M[9] == 3)
@constraint(model, M[19] == 5)
@constraint(model, M[20] == 4)
@constraint(model, M[21] == 3)
@constraint(model, M[29] == 2)
@constraint(model, M[31] == 1)
@constraint(model, conN[i = 1:36, j = 1:36], N[i,j] == A1[i,j]*(1 - Y[i,j]))
#@constraint(model, conY[i = 1:36, j = 1:36], Y[i,j] >= (M[i] - M[j])*0.2)

@constraint(model, con1[i = 1:36], M[i] == )

@constraint(model, sum(N[1, 1:36]) == 1)
@constraint(model, sum(N[3, 1:36]) == 1)
@constraint(model, sum(N[5, 1:36]) == 1)
@constraint(model, sum(N[6, 1:36]) == 1)
@constraint(model, sum(N[7, 1:36]) == 1)
@constraint(model, sum(N[8, 1:36]) == 1)
@constraint(model, sum(N[9, 1:36]) == 1)
@constraint(model, sum(N[19, 1:36]) == 1)
@constraint(model, sum(N[20, 1:36]) == 1)
@constraint(model, sum(N[21, 1:36]) == 1)
@constraint(model, sum(N[29, 1:36]) == 1)
@constraint(model, sum(N[31, 1:36]) == 1)

# Função objetivo
@objective(model,Max,sum(N))

#model